# Scraping API from _techinasia.com/jobs_

## Tools & Library: Python, Postman, Request

**Disclaimer:** This scraping no purpose for sale at all.

### 1. Inisialisasi Library
**requests**: untuk melakukan request HTTP/S

**pandas**: mengatur data dari raw .json

**json**: menyimpan raw .json dari response

In [41]:
import pandas as pd
import numpy as np
import requests
import json

from datetime import datetime

### 2. Inisialisasi URL, Payload, dan Headers untuk keperluan request.
**url**: URL dari API yang ingin di ambil datanya

**payload**: merupakan body yang dikirim saat melakukan request, isi dari body terdapat parameter untuk mengambil data secara spesifik, seperti pekerjaan apa yang dicari, seberapa banyak data tersebut, pekerjaan dari kota mana yang ingin diambil, dan banyak lainnya.

Maksimal dari request ini adalah 999/1000 data, lebih dari itu tidak terpanggil lagi.

In [42]:
url = "https://219wx3mpv4-dsn.algolia.net/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20vanilla%20JavaScript%203.30.0%3BJS%20Helper%202.26.1&x-algolia-application-id=219WX3MPV4&x-algolia-api-key=b528008a75dc1c4402bfe0d8db8b3f8e"

payload = "{\r\n    \"requests\": [\r\n        {\r\n            \"indexName\": \"job_postings\",\r\n            \"params\": \"query=data&hitsPerPage=1000&maxValuesPerFacet=1000&page=0&facets=%5B%22*%22%2C%22city.work_country_name%22%2C%22position.name%22%2C%22industries.vertical_name%22%2C%22experience%22%2C%22job_type.name%22%2C%22is_salary_visible%22%2C%22has_equity%22%2C%22currency.currency_code%22%2C%22salary_min%22%2C%22taxonomies.slug%22%5D&tagFilters=&facetFilters=%5B%5B%22city.work_country_name%3AIndonesia%22%5D%5D\"\r\n        }\r\n    ]\r\n}"

In the payload above, there is the following code:

``query=`` 
*This is used to enter the keywords of the job being searched for. Leave it blank if you want to see all data.*

``hitsPerPage=1000&maxValuesPerFacet=1000`` 
*It is used to retrieve how much data.*

### 3. Melakukan request ke URL yang sudah di inisialisasi sebelumnya dan mengirimkan data sebagai payload.

``status_code``: Melakukan pengecekan apakah statusnya 200 (Berhasil)

``raw``: Variabel untuk menampung raw data dari response

Disini saya menyimpan `raw` data tersebut sebagai json sebelum di lakukan penyaringan maupun manipulasi tipe data, dengan sedikit memberi indent untuk dapat dibaca dengan baik. Karena jika tidak diberi indent, datanya susah untuk di baca.

Dan, saya melakukan pengecekan, apabila statusnya tidak `200 (Tidak berhasil)`, maka tidak perlu di lanjutkan kode setelahnya.

In [43]:
# Melakukan request
# Kenapa post?
# Techinasia sepertinya menggunakan GraphQL sehingga saat melakukan request metodenya POST
response = requests.request("POST", url, data=payload)
print(response.status_code)

if response.status_code == 200:
    # Variabel `raw` sebagi penampungan response
    raw = json.loads(response.text)

    # Menyipan `raw` file
    with open('raw.json', 'w') as r:
        r.write(json.dumps(raw, indent=4))


200


### 4. Mengolah `raw` data

Karena response tersebut berupa `.json`, maka disini saya menggunakan `looping` untuk melakukan penyaringan data kemudian di masukan ke dalam variabel `jobs` yang telah di inisialisasikan sebelumnya. Saya hanya akan mengambil beberapa `key` saja; `Judul pekerjaan, tipe pekerjaan, nama perusahaan, industri perusahaan, kota perusahaan, negara perusahaan, rata-rata - minimal dan maksimal gaji, pengalaman (minimal dan maksimal), keterampilan, dan tanggal listing tersebut di posting.`

Dan, saya akan melakukan pengubahan tipe data pada `publised`, dimana awalnya berupa `string`, kemudian di covert ke `datatime` lalu di ambil hanya tanggalnya saja. Untuk bagian `salary avg, salary min, salary max` juga dilakukan perubahan data, dilakukan untuk memudahkan pengolahan selanjutnya oleh pengguna data ini nantinya, dimana apabila `salary` tersebut kosong, dari `raw data` akan memberikan value `0`, maka perlu perubahan menjadi `NaN` dengan bantuan `NumPy`.

Jika Anda lihat pada kode dibawah ada `if salary > 100` itu karena terkadang ada job list yang salarynya hanya menulis `1, 10` dan sebagainya.

In [44]:
# Inisialisasi List untuk menampung hasil looping
jobs = []

for item in raw['results'][0]['hits']:
    
    # Looping untuk menjadikan 'job_skills' sebagai List, agar memudahkan saat membaca maupun mengolahnya
    skills = [i['name'] for i in item['job_skills']]

    # Convert 'Published' menjadi tipe data `date`
    dt = datetime.strptime(item['published_at'], "%Y-%m-%d %H:%M:%S")
    published = dt.strftime("%Y-%m-%d")

    # Inisialisasi variabel salary untuk memfilter data yang ngasal
    salary = item['salary_avg']
    
    if salary > 100:
        min_salary = item['salary_min']
        max_salary = item['salary_max']
        salary = salary
    else:
        min_salary = np.nan
        max_salary = np.nan
        salary = np.nan

    # Mengatur kolom untuk DataFrame
    job = {
            'Title': item['title'],
            'Job Type': item['job_type']['name'],
            'Company Name': item['company']['name'],
            'Company Industry': item['industries'][0]['name'],
            'Company City': item['city']['name'],
            'Company Country': item['city']['country_name'],
            'Salary Avg': salary,
            'Salary Min': min_salary,
            'Salary Max': max_salary,
            'Experience': item['experience'],
            'Min Experience': item['experience_min'],
            'Max Experience': item['experience_max'],
            'Skills': skills,
            'Published': published
        }
    # Memasukan hasil filter ke dalam List `jobs`
    jobs.append(job)

# Menjadikan List tersebut sebagai DataFrame, agar lebih mudah di olah
# Juga, lebih mudah di lakukan export, seperti .csv, .json
df = pd.DataFrame(jobs)

### 5. Menyimpan data ke .json

Disini saya akan menjadikan 2 tipe `orient` untuk `.json`nya, hasilnya dapat dilihat pada masing-masing file.

In [45]:
df.to_json('data_table.json', orient='table')
df.to_json('data_index.json', orient='index')